In [2]:
import json
from sklearn.preprocessing import MinMaxScaler
import numpy as np

with open('/Users/jasper/Operation Research/Final_Project/schoolStationList.json') as station_file:
    station_info = json.load(station_file)
print(station_info[0]["bemp"])
bike_ava = np.empty((70,1), dtype=float)
for i in range(len(station_info)):
    total = station_info[i]["tot"]
    bemp = station_info[i]["bemp"]
    bike_ava[i,0] = 1-(bemp/total)
print(bike_ava)

29.428571428571427
[[0.7027417 ]
 [0.15714286]
 [0.06666667]
 [0.05882353]
 [0.12142857]
 [0.05854801]
 [0.05279503]
 [0.20833333]
 [0.01428571]
 [0.23571429]
 [0.06878307]
 [0.00510204]
 [0.25714286]
 [0.75396825]
 [0.10357143]
 [0.26190476]
 [0.88095238]
 [0.1042471 ]
 [0.4375    ]
 [0.23015873]
 [0.55714286]
 [0.23333333]
 [0.0952381 ]
 [0.07453416]
 [0.71428571]
 [0.13214286]
 [0.01428571]
 [0.96765499]
 [0.46313364]
 [0.01843318]
 [0.05820106]
 [0.41758242]
 [0.2406015 ]
 [0.18095238]
 [0.65306122]
 [0.07142857]
 [0.06428571]
 [0.74458874]
 [0.        ]
 [0.05952381]
 [0.        ]
 [0.69047619]
 [0.05291005]
 [0.01020408]
 [0.        ]
 [0.15      ]
 [0.20748299]
 [0.44322344]
 [0.24285714]
 [0.9702381 ]
 [0.05882353]
 [0.02380952]
 [0.        ]
 [0.13214286]
 [0.07142857]
 [0.75      ]
 [0.        ]
 [0.16541353]
 [0.18571429]
 [0.09090909]
 [0.07142857]
 [0.01587302]
 [0.24285714]
 [0.04081633]
 [0.06285714]
 [0.08571429]
 [0.07857143]
 [0.        ]
 [0.37857143]
 [0.70634921]]


In [3]:
from gurobipy import *
import pandas as pd

distance_matrix = pd.read_csv('/Users/jasper/Operation Research/Final_Project/distance_matrix.csv')

nodes = []
for i in range(len(distance_matrix)):
    nodes.append(str(i+1))
distance = {}
for i in range(distance_matrix.shape[0]):
    for j in range(distance_matrix.shape[1]):
        distance[(str(i+1),str(j+1))] = distance_matrix.iloc[i,j]*bike_ava.transpose[0,j]

print(distance)
arcs, distance = multidict(distance)
# Create optimization model
m = Model('TSP')
# Create variables
x = {}
u = {}
for i,j in arcs:
    x[i,j] = m.addVar(obj=distance[i,j], vtype = 'B',
       name='x_%s%s' % (i, j))
N = len(nodes)
for i in nodes:
    if i != nodes[N-1]:
        u[i] = m.addVar(obj=0, name='u_%s' % i)
m.update()
# Constraint for sum of incoming links to j
for j in nodes:
     m.addConstr(quicksum(x[i,j]
               for i in nodes if i != j) == 1,
                    'incom_%s' % (j))
# Constraint for sum of outgoing links from i
for i in nodes:
     m.addConstr(quicksum(x[i,j]
               for j in nodes if i != j) == 1,
                    'outgo_%s' % (i))
# Subtour elimination constraints
for i,j in arcs:
    if i != nodes[N-1] and j != nodes[N-1]:
       m.addConstr(u[i] - u[j] + N*x[i,j] <= N-1,
                'subtour_%s_%s' % (i, j))
# Compute optimal solution
m.optimize()
# Print solution
route = []
if m.status == GRB.Status.OPTIMAL:
    print('objective: %f' % m.ObjVal)
    solution = m.getAttr('x', x)
    for i,j in arcs:
         if solution[i,j] > 0:
                print('%s -> %s: %g' % (i, j, solution[i,j]))
                route.append([i,j])
print(route)

{('1', '1'): 0.0, ('1', '2'): 671.1183261183262, ('1', '3'): 781.4487734487735, ('1', '4'): 869.2914862914863, ('1', '5'): 893.8874458874459, ('1', '6'): 786.3679653679654, ('1', '7'): 778.6378066378066, ('1', '8'): 965.5670995670996, ('1', '9'): 860.8585858585859, ('1', '10'): 839.0735930735931, ('1', '11'): 764.5829725829726, ('1', '12'): 659.8744588744589, ('1', '13'): 582.5728715728716, ('1', '14'): 64.65223665223665, ('1', '15'): 694.3088023088023, ('1', '16'): 29.515151515151516, ('1', '17'): 40.75901875901876, ('1', '18'): 278.988455988456, ('1', '19'): 369.64213564213566, ('1', '20'): 318.34199134199133, ('1', '21'): 392.83261183261186, ('1', '22'): 304.989898989899, ('1', '23'): 796.909090909091, ('1', '24'): 236.12121212121212, ('1', '25'): 271.961038961039, ('1', '26'): 197.47041847041848, ('1', '27'): 356.2900432900433, ('1', '28'): 43.56998556998557, ('1', '29'): 477.8643578643579, ('1', '30'): 358.39826839826844, ('1', '31'): 584.6810966810967, ('1', '32'): 524.2453102453

In [4]:
route = pd.DataFrame(route)
route.to_csv('/Users/jasper/Operation Research/Final_Project/route_with_weight.csv', index=False, header=False)